In [69]:
import numpy as np
import pandas as pd
import os
import cv2
import datetime
import pandas as pd
import random

from matplotlib import pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Conv2DTranspose, Reshape, LeakyReLU, LayerNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


def plot_losses(history):
    plt.rcParams['figure.figsize'] = [20, 5]
    f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)

    ax1.set_title('Losses')
    ax1.set_xlabel('epoch')
    ax1.legend(loc="upper right")
    ax1.grid()
    ax1.plot(history['loss'], label='Training loss')
    ax1.plot(history['val_loss'], label='Validation loss')
    ax1.legend()

    ax2.set_title('Accuracy')
    ax2.set_xlabel('epoch')
    ax2.legend(loc="upper right")
    ax2.grid()
    ax2.plot(history['accuracy'], label='Training accuracy')
    ax2.plot(history['val_accuracy'], label='Validation accuracy')
    ax2.legend()

    plt.show()

def plot_resultados(model, carpeta, height=64, width=64, n=4):
    """
    Muestra comparaciones entre imágenes originales y reconstruidas por el autoencoder.
    """
    # Seleccionar imágenes aleatorias
    archivos = os.listdir(carpeta)
    archivos_img = random.sample(archivos, n)

    # Cargar y normalizar las imágenes
    imgs_originales = []
    for nombre in archivos_img:
        img = load_img(os.path.join(carpeta, nombre), target_size=(height, width))
        img_array = img_to_array(img) / 255.0  # normalizar a [0,1]
        imgs_originales.append(img_array)

    imgs_originales = np.array(imgs_originales)

    # Reconstruir con el modelo
    imgs_reconstruidas = model.predict(imgs_originales)

    # Mostrar resultados
    plt.figure(figsize=(12, 6))
    for i in range(n):
        # Imagen original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(imgs_originales[i])
        ax.set_title("Original")
        ax.axis("off")

        # Imagen reconstruida
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(imgs_reconstruidas[i])
        ax.set_title("Reconstruida")
        ax.axis("off")

    plt.tight_layout()
    plt.show()

# VAE

## Defining Hiperparameters

In [70]:
import os

os.listdir("/kaggle/input")

['images']

In [71]:
height, width = 64, 64 # Image size, to fit in the competition size
latent_dim = 256

image_dir = '/kaggle/input'
model_path = None # "models/"

## Loss

In [72]:
def reconstruction_loss(y_true, y_pred):
    return tf.reduce_sum(tf.keras.losses.binar(y_true, y_pred), axis=(1,2,3))
def kl_loss(mu, log_var):
    return -0.5 * tf.reduce_sum(1 + log_var - tf.square(mu) - tf.exp(log_var), axis=1)


## Model

In [75]:
class Sampling(layers.Layer):
    def call(self, inputs):
        mu, log_var = inputs
        sigma = tf.exp(0.5 * log_var)
        epsilon = tf.random.normal(shape=tf.shape(mu))
        z = mu + sigma * epsilon

        # KL divergence
        kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(1 + log_var - tf.square(mu) - tf.exp(log_var), axis=1)
        )
        beta = 1/ latent_dim * 50  
        self.add_loss(beta * kl_loss)

        
        return z

In [76]:
input_img = keras.Input(shape=(height, width, 3))

# Encoder
x_enc = Conv2D(16, (3, 3), strides=2, padding="same")(input_img)
x_enc = LeakyReLU(alpha=0.2)(x_enc)
x_enc = Conv2D(32, (3, 3), strides=2, padding="same")(x_enc)
x_enc = LeakyReLU(alpha=0.2)(x_enc)
x_enc = LayerNormalization()(x_enc)
x_enc = Conv2D(64, (3, 3), strides=2, padding="same")(x_enc)
x_enc = LeakyReLU(alpha=0.2)(x_enc)
x_enc = Conv2D(128, (3, 3), strides=2, padding="same")(x_enc)
x_enc = LeakyReLU(alpha=0.2)(x_enc)
x_enc = LayerNormalization()(x_enc)
x_enc = Conv2D(256, (3, 3), strides=2, padding="same")(x_enc)
x_enc = LeakyReLU(alpha=0.2)(x_enc)
x_enc = Dropout(0.1)(x_enc)
x_enc = Conv2D(512, (3, 3), strides=2, padding="same")(x_enc)
x_enc = LeakyReLU(alpha=0.2)(x_enc)
x_enc = LayerNormalization()(x_enc)
shape_before_flattening = keras.backend.int_shape(x_enc)[1:]
x_enc = Flatten()(x_enc)
x_enc = Dense(512, activation="relu")(x_enc)

# VAE: media y log-varianza
mu = Dense(latent_dim, name="mu")(x_enc)
log_var = Dense(latent_dim, name="log_var")(x_enc)

# Muestreo
z = Sampling()([mu, log_var])


# Decoder
x_dec = Dense(np.prod(shape_before_flattening), activation="relu")(z)
x_dec = Reshape(shape_before_flattening)(x_dec)
x_dec = LayerNormalization()(x_dec)
x_dec = Conv2DTranspose(512, (3, 3), activation="relu", strides=2, padding="same")(x_dec)
x_dec = Conv2DTranspose(256, (3, 3), activation="relu", strides=2, padding="same")(x_dec)
x_dec = LayerNormalization()(x_dec)
x_dec = Conv2DTranspose(128, (3, 3), activation="relu", strides=2, padding="same")(x_dec)
x_dec = Conv2DTranspose(64, (3, 3), activation="relu", strides=2, padding="same")(x_dec)
x_dec = LayerNormalization()(x_dec)
x_dec = Conv2DTranspose(32, (3, 3), activation="relu", strides=2, padding="same")(x_dec)
x_dec = Conv2DTranspose(16, (3, 3), activation="relu", strides=2, padding="same")(x_dec)
decoded = Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x_dec)

# reconstruction_loss = tf.reduce_mean(
#     tf.reduce_sum(
#         tf.keras.losses.binary_crossentropy(input_img, decoded), axis=(1,2,3)
#     )
# )
# vae.add_metric(reconstruction_loss, name="reconstruction_loss")


We define the Variational autoencoder, encoder and decoder

In [77]:
if not model_path:
    vae = keras.Model(input_img, decoded)
    encoder = keras.Model(input_img, z)
    decoder = keras.Model(z, decoded)
else:
    vae = keras.load_model(model_path + "/vae.keras")
    encoder = keras.Model(model_path + "/encoder.keras")
    decoder = keras.Model(model_path + "/decoder.keras")
    
vae.summary()

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_38 (Conv2D)  │ (None, 32, 32,    │        448 │ input_layer_8[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_14      │ (None, 32, 32,    │          0 │ conv2d_38[0][0]   │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_39 (Conv2D)  │ (None, 16, 16,    │      4,640 │ leaky_re_lu_14[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_15      │ (None, 16, 16,    │          0 │ conv2d_39[0][0]   │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 16, 16,    │         64 │ leaky_re_lu_15[0… │
│ (LayerNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_40 (Conv2D)  │ (None, 8, 8, 64)  │     18,496 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_16      │ (None, 8, 8, 64)  │          0 │ conv2d_40[0][0]   │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 4, 4, 128) │     73,856 │ leaky_re_lu_16[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_17      │ (None, 4, 4, 128) │          0 │ conv2d_41[0][0]   │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 4, 4, 128) │        256 │ leaky_re_lu_17[0… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_42 (Conv2D)  │ (None, 2, 2, 256) │    295,168 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_18      │ (None, 2, 2, 256) │          0 │ conv2d_42[0][0]   │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 2, 2, 256) │          0 │ leaky_re_lu_18[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_43 (Conv2D)  │ (None, 1, 1, 512) │  1,180,160 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_19      │ (None, 1, 1, 512) │          0 │ conv2d_43[0][0]   │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 1, 512) │      1,024 │ leaky_re_lu_19[0… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 512)       │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 512)       │    262,656 │ flatten_5[0][0] 

 Total params: 6,164,739 (23.52 MB)

 Trainable params: 6,164,739 (23.52 MB)

 Non-trainable params: 0 (0.00 B)

In [80]:
opt = keras.optimizers.Adam(learning_rate=1e-3)

# Loss combinada BCE + MSE
def reconstruction_loss(y_true, y_pred):
    # BCE promedio por pixel
    bce = tf.reduce_mean(keras.losses.binary_crossentropy(y_true, y_pred))
    
    # MSE promedio por pixel
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    
    return 0.5 * bce + 0.5 * mse



vae.compile(
    optimizer=opt,
    loss="mse",
    metrics=["mse"]
)


## ImageDataGenerator

To fit with the competition, we will reshape images to 64x64

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create training and validation generators using the same seed
train_generator = datagen.flow_from_directory(
    image_dir,
    target_size=(height, width),  
    batch_size=128,
    class_mode='input',
    subset='training',
    shuffle=True,
    seed=2004  
)

val_generator = datagen.flow_from_directory(
    image_dir,
    target_size=(height, width),
    batch_size=128,
    class_mode='input',
    subset='validation',
    shuffle=False,
    seed=2004
)

## Train

In [ ]:
# Fit the autoencoder
h = vae.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

Epoch 1/20
398/398 ━━━━━━━━━━━━━━━━━━━━ 90s 188ms/step - loss: 0.1086 - mse: 0.0670 - val_loss: 0.0667 - val_mse: 0.0667
Epoch 2/20
398/398 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 0.0667 - mse: 0.0667

In [ ]:
plot_losses(h.history)

In [ ]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
## Make sure everything saves correctly
os.makedirs("models", exist_ok=True)
model_path = f"models/VAE_{timestamp}.keras"

# Save the model
os.makedirs(f"models/{model_path}", exist_ok=True)
vae.save(f"models/{model_path}/vae.keras")
encoder.save(f"models/{model_path}/encoder.keras")
decoder.save(f"models/{model_path}/decoder.keras")

In [ ]:
plot_resultados(vae, "/kaggle/input/images")